In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from sklearn.metrics.pairwise import euclidean_distances

In [3]:
# notebooks/example_notebook.ipynb
import sys
import os
sys.path.append(os.path.abspath('../scripts'))

In [4]:
from load_data import load_data_from_postgres, load_data_using_sqlalchemy
from sql_queries import execute_telecom_queries

In [5]:
# Load environment variables from .env file
load_dotenv()
# Fetch database connection parameters from environment variables
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [6]:
# Define your SQL query
query = "SELECT * FROM xdr_data;"

# Load data from PostgreSQL using SQLAlchemy
df = load_data_using_sqlalchemy(query)

# Display the first few rows of the dataframe
if df is not None:
    print("Successfully loaded the data")
else:
    print("Failed to load data.")

Successfully loaded the data


In [7]:
# Replace missing values
for col in ['Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)', 'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)', 'Handset Type']:
    if df[col].dtype == 'object':
        # Replace NaN with mode for categorical columns
        df[col] = df[col].fillna(df[col].mode()[0])
    else:
        # Replace NaN with mean for numeric columns
        df[col] = df[col].fillna(df[col].mean())

# Aggregate per customer
aggregated_experience = df.groupby('MSISDN/Number').agg({
    'TCP DL Retrans. Vol (Bytes)': 'mean',
    'TCP UL Retrans. Vol (Bytes)': 'mean',
    'Avg RTT DL (ms)': 'mean',
    'Avg RTT UL (ms)': 'mean',
    'Avg Bearer TP DL (kbps)': 'mean',
    'Avg Bearer TP UL (kbps)': 'mean',
    'Handset Type': lambda x: x.mode().iloc[0]  # Handle mode safely
}).reset_index()

# Rename columns for clarity
aggregated_experience.rename(columns={
    'TCP DL Retrans. Vol (Bytes)': 'Avg TCP DL Retransmission',
    'TCP UL Retrans. Vol (Bytes)': 'Avg TCP UL Retransmission',
    'Avg RTT DL (ms)': 'Avg RTT DL',
    'Avg RTT UL (ms)': 'Avg RTT UL',
    'Avg Bearer TP DL (kbps)': 'Avg Throughput DL',
    'Avg Bearer TP UL (kbps)': 'Avg Throughput UL'
}, inplace=True)


In [9]:
# Step 1: Aggregate Metrics
aggregated_metrics = df.groupby('MSISDN/Number').agg({
    'TCP DL Retrans. Vol (Bytes)': 'mean',
    'TCP UL Retrans. Vol (Bytes)': 'mean',
    'Avg RTT DL (ms)': 'mean',
    'Avg RTT UL (ms)': 'mean',
    'Avg Bearer TP DL (kbps)': 'mean',
    'Avg Bearer TP UL (kbps)': 'mean',
    'Handset Type': 'first'  # Assuming a single handset type per user
}).reset_index()

# Rename columns for clarity
aggregated_metrics.rename(columns={
    'TCP DL Retrans. Vol (Bytes)': 'Avg TCP DL Retransmission',
    'TCP UL Retrans. Vol (Bytes)': 'Avg TCP UL Retransmission',
    'Avg RTT DL (ms)': 'Avg RTT DL',
    'Avg RTT UL (ms)': 'Avg RTT UL',
    'Avg Bearer TP DL (kbps)': 'Avg Throughput DL',
    'Avg Bearer TP UL (kbps)': 'Avg Throughput UL'
}, inplace=True)




In [13]:
print(aggregated_metrics.columns)


Index(['MSISDN/Number', 'Avg TCP DL Retransmission',
       'Avg TCP UL Retransmission', 'Avg RTT DL', 'Avg RTT UL',
       'Avg Throughput DL', 'Avg Throughput UL', 'Handset Type'],
      dtype='object')


## Compute Engagement and Experience Scores

In [14]:
# Assuming 'Avg Throughput DL' and 'Avg Throughput UL' are in kbps, you can add them to get total traffic
aggregated_metrics['Total Traffic'] = aggregated_metrics['Avg Throughput DL'] + aggregated_metrics['Avg Throughput UL']


In [24]:
# Assuming you are working with available metrics for experience and engagement clustering
# Engagement clustering
engagement_features = aggregated_metrics[['Avg RTT DL', 'Avg Throughput DL', 'Avg TCP DL Retransmission']]
scaler = StandardScaler()
normalized_engagement_features = scaler.fit_transform(engagement_features)

# Apply KMeans clustering
engagement_kmeans = KMeans(n_clusters=3, random_state=42)
engagement_clusters = engagement_kmeans.fit(normalized_engagement_features)

# Print the cluster centers for engagement clustering
print("Engagement Cluster Centers:\n", engagement_kmeans.cluster_centers_)


Engagement Cluster Centers:
 [[-7.02421533e-02  1.82629432e+00 -4.71266588e-03]
 [ 1.44567817e-02 -3.79256805e-01 -3.95350593e-02]
 [-2.31760126e-02  2.48497102e+00  2.25510628e+01]]


In [25]:
# Experience clustering (using the same features for example)
experience_features = aggregated_metrics[['Avg RTT DL', 'Avg Throughput DL', 'Avg TCP DL Retransmission']]
normalized_experience_features = scaler.fit_transform(experience_features)

# Apply KMeans clustering
experience_kmeans = KMeans(n_clusters=3, random_state=42)
experience_clusters = experience_kmeans.fit(normalized_experience_features)

# Print the cluster centers for experience clustering
print("Experience Cluster Centers:\n", experience_kmeans.cluster_centers_)

Experience Cluster Centers:
 [[-7.02421533e-02  1.82629432e+00 -4.71266588e-03]
 [ 1.44567817e-02 -3.79256805e-01 -3.95350593e-02]
 [-2.31760126e-02  2.48497102e+00  2.25510628e+01]]


In [26]:
# Assuming we are using engagement centroids for one cluster (e.g., the first cluster)
engagement_centroid = engagement_kmeans.cluster_centers_[0]  # First cluster centroid
experience_centroid = experience_kmeans.cluster_centers_[0]  # First cluster centroid

# Calculate the Euclidean distance for each user from the centroids
aggregated_metrics['Engagement Score'] = np.linalg.norm(aggregated_metrics[['Avg RTT DL', 'Avg Throughput DL', 'Avg TCP DL Retransmission']].values - engagement_centroid, axis=1)
aggregated_metrics['Experience Score'] = np.linalg.norm(aggregated_metrics[['Avg TCP DL Retransmission', 'Avg RTT DL', 'Avg Throughput DL']].values - experience_centroid, axis=1)

# View the scores
print(aggregated_metrics[['Engagement Score', 'Experience Score']].head())


   Engagement Score  Experience Score
0      2.080991e+07      2.080991e+07
1      2.080991e+07      2.080991e+07
2      2.080991e+07      2.080991e+07
3      1.087203e+03      1.087491e+03
4      1.507979e+07      1.507979e+07


## Satisfaction Score

In [32]:
# Calculate the satisfaction score as the average of engagement and experience scores
aggregated_metrics['Satisfaction Score'] = (aggregated_metrics['Engagement Score'] + aggregated_metrics['Experience Score']) / 2

# Sort by satisfaction score and get top 10 satisfied customers
top_10_satisfied_customers = aggregated_metrics[['MSISDN/Number', 'Satisfaction Score']].sort_values(by='Satisfaction Score', ascending=False).head(10)
print(top_10_satisfied_customers)


       MSISDN/Number  Satisfaction Score
85785   3.369858e+10        4.289488e+09
32273   3.365871e+10        4.288060e+09
58026   3.366491e+10        4.268432e+09
31942   3.365863e+10        4.254644e+09
62049   3.366613e+10        4.211189e+09
70079   3.366877e+10        4.166591e+09
43629   3.366131e+10        4.131046e+09
48341   3.366240e+10        4.117753e+09
64113   3.366682e+10        3.968072e+09
98211   3.376264e+10        3.785295e+09


## Predict Satisfaction Score

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Define the features (engagement and experience scores) and target (satisfaction score)
X = aggregated_metrics[['Engagement Score', 'Experience Score']]
y = aggregated_metrics['Satisfaction Score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the Random Forest model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict satisfaction scores
y_pred = model.predict(X_test)

# Evaluate the model performance (e.g., using R^2 score)
print(f'R^2 score: {r2_score(y_test, y_pred)}')


R^2 score: 0.9998991464740772


## Cluster Satisfaction

In [35]:
# Perform K-Means clustering (k=2) on the engagement and experience scores
from sklearn.cluster import KMeans

# Combine engagement and experience scores for clustering
X_cluster = aggregated_metrics[['Engagement Score', 'Experience Score']]

# Apply K-Means clustering (k=2)
kmeans = KMeans(n_clusters=2, random_state=42)
aggregated_metrics['Satisfaction Cluster'] = kmeans.fit_predict(X_cluster)

# Aggregate average satisfaction and experience scores per cluster
cluster_avg_scores = aggregated_metrics.groupby('Satisfaction Cluster')[['Satisfaction Score', 'Engagement Score', 'Experience Score']].mean()
print(cluster_avg_scores)


                      Satisfaction Score  Engagement Score  Experience Score
Satisfaction Cluster                                                        
0                           1.682820e+07      1.682820e+07      1.682820e+07
1                           2.224330e+09      2.224330e+09      2.224330e+09


## 5: Export to Postgres

In [41]:
from sqlalchemy import text

# Create a connection string for PostgreSQL
db_connection_str = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(db_connection_str)

# Save the final table to PostgreSQL (replace 'user_scores' with your table name)
aggregated_metrics[['MSISDN/Number', 'Engagement Score', 'Experience Score', 'Satisfaction Score']].to_sql('user_scores', con=engine, if_exists='replace', index=False)

# Query to check if the data is inserted correctly
with engine.connect() as connection:
    query = text('SELECT * FROM user_scores LIMIT 10;')  # Wrap the query with text()
    result = connection.execute(query).fetchall()

# Display result
print(result)


[(33601001722.0, 20809914.27513532, 20809914.34066387, 20809914.307899594), (33601001754.0, 20809914.27512755, 20809914.340658523, 20809914.307893038), (33601002511.0, 20809914.27539584, 20809914.340919543, 20809914.30815769), (33601007832.0, 1087.2032712623193, 1087.4906853526488, 1087.346978307484), (33601008617.0, 15079785.663573071, 15079785.731547737, 15079785.697560403), (33601010682.0, 10406337.819069417, 10406337.885248734, 10406337.852159075), (33601011634.0, 10454116.207657216, 10454116.276771015, 10454116.242214115), (33601011959.0, 796.8552946171471, 798.476335794216, 797.6658152056816), (33601014694.0, 20809914.27537693, 20809914.340899795, 20809914.308138363), (33601020306.0, 20809914.275269944, 20809914.34080056, 20809914.308035254)]
